In [1]:
import pandas as pd
import numpy as np

## 1. ОП проект счёт

In [2]:
ss_project_invoice = pd.read_excel('files/ОП_Проект_Счет.xlsx')

In [3]:
ss_project_invoice.head()

,Обособленное подразделение,Проект,Код проекта,Расчетные счета
0,"Газопровод-отвод и ГРС ""Селятино"" со строитель...",СМР по реконструкции газопровода-отвода и ГРС ...,2008,40702810500000049708
1,Корвет-Киренск,СМР Этап 1Линейн.часть газопровода в составе с...,2002,40702810400000022472
2,Корвет-Киренск,"СМР Этап 5.1Компрессорная станция КС-1""Салдыке...",2003,40702810100000024068
3,Корвет-Киренск,СМР по организ.рельефа УКПГ-3 в сост. стройки ...,2005,40702810900000022477
4,Корвет-Киренск,СМР промбаза Нючакан по мульчир. на обьекте в ...,2005,40702810300000028628


## 2. статьи затрат 

In [4]:
cost_items = pd.read_excel('files/Статьи_затрат.xlsx', header=None)
cost_items.columns = ['Счет', 'Статья затрат']

In [5]:
cost_items.head()

,Счет,Статья затрат
0,60,Расчеты с поставщиками
1,62,Расчеты с покупателями
2,68,Расчеты по налогам
3,69,Расчеты по страховым взносам
4,70,Расчеты по заработной плате


## код_2008_Кт51

In [6]:
ct_51 = pd.read_excel('files/Селятино/код_2008_Кт51.xlsx', skiprows=8)[:-1] # без первых строк и последней 

In [7]:
# удаляем колонки 
ct_51 = ct_51.drop(['Unnamed: 6', 'Счет.1', 'Unnamed: 8'], axis=1) 
# Оставляю колонку Unnamed: 5 для последующего просчета -Сумма расхода

In [8]:
# Делим столбец -3
ct_51['Имя'] = ct_51['Unnamed: 3'].str.split('\n', 1).str[0]
ct_51['Расчётный счёт'] = ct_51['Unnamed: 3'].str.split('\n', 1).str[1].str[:20]

# Колонка с со счётом - только первые два символа
ct_51['Счет'] = ct_51['Счет'].str[:2].astype(int)

In [9]:
ct_51.head(2)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Счет,Unnamed: 5,Имя,Расчётный счёт
0,19.08.2022,Списание с расчетного счета ОПБП-009148 от 19....,Корвет-Киренск\nАТП 2020 ООО\nАТП-69П от 01.10...,"Корвет-Киренск\n40702810500000049708, БАНК ГПБ...",60,49012.78,Корвет-Киренск,40702810500000049708
1,19.08.2022,Списание с расчетного счета ОПБП-009094 от 19....,Уткин Алексей Геннадьевич\n<...>,"Корвет-Верейская\n40702810500000049708, БАНК Г...",70,92500.00,Корвет-Верейская,40702810500000049708


## удаляем строки со значениями организаций в столбце 'аналитика ДТ', но без 'фин' в столбце 'Документ'

In [10]:
# Список организаций состоящих в Корвет
values_to_remove = ['АТП 2020', 'СТРОЙ-ТРАК СЕРВИС', 'САХАСТРОЙДОРТРАНС']
pattern = '|'.join(values_to_remove)
# Список фин
values_to_stay = ['Фин', 'фин'] 
pattern_stay = '|'.join(values_to_stay)

In [11]:
# Создаем таблицу где нет значения фин
without_fin = ct_51.loc[~ct_51["Unnamed: 1"].str.contains(pattern_stay, case=False)]

In [12]:
# Создаем таблицу где есть значения фин
fin_truth = ct_51.loc[ct_51["Unnamed: 1"].str.contains(pattern_stay, case=False)]

In [13]:
# Удаляем строки со значениями из списка из таблицы с отсутствующим значением фин 
without_fin = without_fin.loc[~without_fin["Unnamed: 2"].str.contains(pattern, case=False)]

In [14]:
# Соединяем таблицы 
ct_51 = pd.concat((without_fin, fin_truth), axis=0)

In [15]:
# Удаляем столбцы 
ct_51 = ct_51.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'], axis=1)

In [16]:
ct_51.head()

,Счет,Unnamed: 5,Имя,Расчётный счёт
1,70,92500.0,Корвет-Верейская,40702810500000049708
3,70,192500.0,Корвет-Верейская,40702810500000049708
4,70,69520.0,Корвет-Верейская,40702810500000049708
7,70,192500.0,Корвет-Верейская,40702810500000049708
8,70,50794.0,Корвет-Верейская,40702810500000049708


In [17]:
# Именуем колонки 
ct_51.columns = ['Счёт расход', 'Сумма расхода', 'ОП затрат', 'Расчетный счет'] 

In [18]:
ct_51.head()

,Счёт расход,Сумма расхода,ОП затрат,Расчетный счет
1,70,92500.0,Корвет-Верейская,40702810500000049708
3,70,192500.0,Корвет-Верейская,40702810500000049708
4,70,69520.0,Корвет-Верейская,40702810500000049708
7,70,192500.0,Корвет-Верейская,40702810500000049708
8,70,50794.0,Корвет-Верейская,40702810500000049708


 ## объединяем таблицы - Пункт 4

In [19]:
# соединяем 
ct_51 = ct_51.merge(cost_items, how='left', left_on=['Счёт расход'], right_on=['Счет']) 

In [20]:
ct_51.head()

,Счёт расход,Сумма расхода,ОП затрат,Расчетный счет,Счет,Статья затрат
0,70,92500.0,Корвет-Верейская,40702810500000049708,70,Расчеты по заработной плате
1,70,192500.0,Корвет-Верейская,40702810500000049708,70,Расчеты по заработной плате
2,70,69520.0,Корвет-Верейская,40702810500000049708,70,Расчеты по заработной плате
3,70,192500.0,Корвет-Верейская,40702810500000049708,70,Расчеты по заработной плате
4,70,50794.0,Корвет-Верейская,40702810500000049708,70,Расчеты по заработной плате


In [21]:
# Заменяем значение в столбце на более корректные - Без ЗУП и пробелов 
ct_51 = ct_51.replace({'ОП затрат' : {'Основное подразделение ЗУП': 'Основное подразделение', 
                              'Основное подразделение  ': 'Основное подразделение'}})

In [22]:
ct_51.head(2)

,Счёт расход,Сумма расхода,ОП затрат,Расчетный счет,Счет,Статья затрат
0,70,92500.0,Корвет-Верейская,40702810500000049708,70,Расчеты по заработной плате
1,70,192500.0,Корвет-Верейская,40702810500000049708,70,Расчеты по заработной плате


In [23]:
# Присоединяем колонку -код проекта 
ct_51 = ct_51.merge(ss_project_invoice, 
                    how='inner', left_on=['ОП затрат'], 
                    right_on=['Обособленное подразделение']) 

In [24]:
ct_51.head(2)

,Счёт расход,Сумма расхода,ОП затрат,Расчетный счет,Счет,Статья затрат,Обособленное подразделение,Проект,Код проекта,Расчетные счета
0,70,92500.0,Корвет-Верейская,40702810500000049708,70,Расчеты по заработной плате,Корвет-Верейская,Корвет-Верейская,4000КВ,NaN
1,70,192500.0,Корвет-Верейская,40702810500000049708,70,Расчеты по заработной плате,Корвет-Верейская,Корвет-Верейская,4000КВ,NaN


In [25]:
# новые столбцы 
ct_51['Проект расхода'] = ct_51['ОП затрат']
ct_51['Код проекта расхода'] = ct_51['Код проекта']
# Переименование столбцов 
ct_51.rename(columns={'ОП затрат': 'ОП', 'Расчетные счета': 'Расчётный счёт', 'Статья затрат': 'Статья расхода'}, 
             inplace=True)

In [26]:
ct_51.head(2)

,Счёт расход,Сумма расхода,ОП,Расчетный счет,Счет,Статья расхода,Обособленное подразделение,Проект,Код проекта,Расчётный счёт,Проект расхода,Код проекта расхода
0,70,92500.0,Корвет-Верейская,40702810500000049708,70,Расчеты по заработной плате,Корвет-Верейская,Корвет-Верейская,4000КВ,NaN,Корвет-Верейская,4000КВ
1,70,192500.0,Корвет-Верейская,40702810500000049708,70,Расчеты по заработной плате,Корвет-Верейская,Корвет-Верейская,4000КВ,NaN,Корвет-Верейская,4000КВ


In [27]:
# Меняем порядок столбцов 
ct_51 = ct_51.reindex(columns=['ОП', 'Проект', 'Код проекта', 'Расчетный счет', 'Счёт расход', 'Статья расхода', 'Проект расхода', 
                               'Код проекта расхода', 'Сумма расхода'])

In [28]:
ct_51.head(2)

,ОП,Проект,Код проекта,Расчетный счет,Счёт расход,Статья расхода,Проект расхода,Код проекта расхода,Сумма расхода
0,Корвет-Верейская,Корвет-Верейская,4000КВ,40702810500000049708,70,Расчеты по заработной плате,Корвет-Верейская,4000КВ,92500.0
1,Корвет-Верейская,Корвет-Верейская,4000КВ,40702810500000049708,70,Расчеты по заработной плате,Корвет-Верейская,4000КВ,192500.0


- В алгоритме указано-
#### Группируем по столбцам:
#### ['ОП', 'Проект', 'Код проекта', 'Расчетный счет', 'Код проекта расхода', 'Проект расхода',
#### 'Счет расход', 'Статья расхода']

# 'Счет расход' - не определила что имеется ввиду. Не указала данное значение в группировке

In [29]:
ct_51 = ct_51.groupby(['ОП', 'Проект', 'Код проекта', 'Расчетный счет', 'Код проекта расхода', 'Проект расхода',
'Счёт расход', 'Статья расхода'])['Сумма расхода'].sum().reset_index()
 

In [30]:
new_row = {'ОП': np.nan, 'Проект': np.nan, 'Код проекта': np.nan, 'Расчетный счет': np.nan, 'Код проекта расхода': np.nan,
           'Проект расхода': np.nan, 'Статья расхода': 'Сумма - ', 'Сумма расхода': ct_51['Сумма расхода'].sum()}
ct_51 = ct_51.append(new_row, ignore_index=True)


/var/folders/1m/r8sjkd3n21s0fjc__2nczm000000gn/T/ipykernel_18702/2685618753.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ct_51 = ct_51.append(new_row, ignore_index=True)


In [31]:
ct_51[:3]

,ОП,Проект,Код проекта,Расчетный счет,Код проекта расхода,Проект расхода,Счёт расход,Статья расхода,Сумма расхода
0,"Газопровод-отвод и ГРС ""Селятино"" со строитель...",СМР по реконструкции газопровода-отвода и ГРС ...,2008,40702810500000049708,2008,"Газопровод-отвод и ГРС ""Селятино"" со строитель...",60.0,Расчеты с поставщиками,24076389.07
1,"Газопровод-отвод и ГРС ""Селятино"" со строитель...",СМР по реконструкции газопровода-отвода и ГРС ...,2008,40702810500000049708,2008,"Газопровод-отвод и ГРС ""Селятино"" со строитель...",62.0,Расчеты с покупателями,1503762.72
2,"Газопровод-отвод и ГРС ""Селятино"" со строитель...",СМР по реконструкции газопровода-отвода и ГРС ...,2008,40702810500000049708,2008,"Газопровод-отвод и ГРС ""Селятино"" со строитель...",68.0,Расчеты по налогам,103295.00


In [33]:
ct_51[-3:]

,ОП,Проект,Код проекта,Расчетный счет,Код проекта расхода,Проект расхода,Счёт расход,Статья расхода,Сумма расхода
47,"Реконструкция ГРС ""Северо-Задонская""",СМР по реконструкции ГРС Северо-Задонская,2009,40702810500000049708,2009,"Реконструкция ГРС ""Северо-Задонская""",60.0,Расчеты с поставщиками,1.053059e+07
48,"Реконструкция ГРС ""Северо-Задонская""",СМР по реконструкции ГРС Северо-Задонская,2009,40702810500000049708,2009,"Реконструкция ГРС ""Северо-Задонская""",76.0,Расчеты с прочими дебиторами/кредиторами,4.130500e+05
49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Сумма -,2.499902e+08


In [32]:
ct_51.to_excel('Отчет.xlsx', sheet_name='Отчет', index=False)